In [1]:
import pandas as pd
from Skeleton import Skeleton,DynamicSkeleton
import nimblephysics as nimble
import numpy as np
import json
import time

s12 = Skeleton('/home/rmhri/markerless-human-perception/src/hpe/src/Biomechanical-Model/BODY12.xml')
# Rajagopal
# s = DynamicSkeleton('/home/rmhri/markerless-human-perception/src/hpe/src/Biomechanical-Model/BODY15_constrained_3D.xml')
# BSM
s = DynamicSkeleton(config='/home/rmhri/markerless-human-perception/src/hpe/src/Biomechanical-Model/BODY15_constrained_3D.xml', \
    osim_file='/media/rmhri/mrom/data/skel_models_v1.1/bsm.osim')# \
    # geometry_dir='/home/rmhri/markerless-human-perception/src/hpe/src/Biomechanical-Model/data/skel_models_v1.1/Geometry/')

kps =  ['RKnee', 'LWrist', 'RHip', 'RShoulder',  'LElbow', 'LHip', 'RElbow', 'RWrist', 'LKnee', 'LShoulder', 'RAnkle', 'LAnkle']
gui = nimble.NimbleGUI()
gui.serve(8080)
# gui.nativeAPI().renderBasis()

with open('tmp/camera-parameters.json') as f:
    parameters = json.load(f)

R = np.array(parameters["extrinsics"]['S9']['60457274']['R'])
T = np.array(parameters["extrinsics"]['S9']['60457274']['t'])
RT = np.vstack([np.hstack([R,T]),np.array([0,0,0,1])])
RT = np.linalg.inv(RT)
R = RT[0:3,0:3]
T = RT[0:3,3].reshape(3,1)

# gui.nativeAPI().renderSkeleton(s._nimble)

# [s._nimble.getJoint(i).getDofName(j) for i in range(s._nimble.getNumJoints()) for j in range(s._nimble.getJoint(i).getNumDofs())]
# j = s._nimble.getJoint(2)

# j.getNumDofs?

Setting len to 0.1
Setting neutral pos to   0
0.7
  0
Setting len to 0.3
Setting neutral pos to    0
-0.6
   0
Setting len to 0.1
Setting neutral pos to   0
0.2
  0
GUIWebsocketServer will start serving a WebSocket server on ws://localhost:8070
Web GUI serving on http://localhost:8080


In [2]:
# gui.nativeAPI().renderSkeleton(s15.nimble_skeleton)
df = pd.read_csv('/home/rmhri/mrom/data/S9_Directions.60457274_gt.csv')
t = 0
Q = []
P = []
s.prob = None
for t in range(0,df.shape[0]):
    target = np.array([np.array(df.loc[t,[l+":X",l+":Y",l+":Z"]].to_numpy()) for l in kps]).squeeze()/1000
    
    # Rotate for beautiful view
    res = np.dot(R,target.reshape(-1,3).transpose()*1000) + T
    Rx = np.array([
        [1,0,0],
        [0,0,1],
        [0,-1,0]
    ])
    res = np.dot(Rx,res)
    target = res.transpose()/1000
    
    # Remove a keypoint
    target[4,:] = np.nan
    
    s12.load_from_numpy(target.reshape(-1,3),kps)
    s.load_from_BODY12(s12)
    s.scale()
    print(s)
    # s.gdIK(1000)
    
    print(np.round(s.to_numpy().reshape(-1,3),2))
    break
    # print(target)
    
    if s.prob is None:
        s.reset_position()
        s.qpIK(1000,100)
    else:
        s.qpIK(1000,0.02)
    
    # Rendering
    gui.nativeAPI().renderSkeleton(s._nimble)
        
    # Plot the measurement
    target = target.reshape(-1,1)
    for i in range(0,int(target.shape[0]/3)):
        gui.nativeAPI().createSphere(key=kps[i], radii=np.array([0.01,0.01,0.01]),pos=target[3*i:3*i+3],color=np.array([0,255,0,0.8]))

    pos = s._nimble.getJointWorldPositions(s.joints)
    pos = s.correct(pos)

    for i in range(0,int(pos.shape[0]/3)):
        gui.nativeAPI().createSphere(key=s.joints[i].getName(), radii=np.array([0.01,0.01,0.01]),pos=pos[3*i:3*i+3],color=np.array([0,0,0,1]))
    Q.append(s._nimble.getPositions())
    P.append(s.to_numpy())
    time.sleep(0.02)
    gui.nativeAPI().createSphere(key="COM", radii=np.array([0.01,0.01,0.01]),pos=s._nimble.getCOM(),color=np.array([1,0,0,1]))
    # print(t,np.round(s._nimble.getBodyScales().reshape(-1,3)[:,0].transpose(),2))
Q = np.array(Q)
P = np.array(P)

Skeleton (format: BODY15, dim: 3)
(Root: [-0.03  1.24 -0.08])
-[USpine: 0.25]-(MidShoulder: [-0.03  1.49 -0.09])
	-[LClavicle: 0.18]-(LShoulder: [ 0.15  1.49 -0.1 ])
		-[LHumerus: nan]-(LElbow: [nan nan nan])
			-[LForearm: nan]-(LWrist: [ 0.67  1.41 -0.01])

	-[RClavicle: 0.18]-(RShoulder: [-0.21  1.48 -0.09])
		-[RHumerus: 0.3]-(RElbow: [-0.5   1.44 -0.11])
			-[RForearm: 0.25]-(RWrist: [-0.74  1.43 -0.04])

-[DSpine: 0.25]-(MidHip: [-0.04  0.99 -0.06])
	-[LPelvis: 0.12]-(LHip: [ 0.09  0.97 -0.06])
		-[LFemur: 0.47]-(LKnee: [ 0.1   0.5  -0.07])
			-[LTibia: 0.47]-(LAnkle: [ 0.11  0.04 -0.16])

	-[RPelvis: 0.12]-(RHip: [-0.16  1.   -0.06])
		-[RFemur: 0.47]-(RKnee: [-0.18  0.53 -0.09])
			-[RTibia: 0.47]-(RAnkle: [-0.19  0.07 -0.17])

[[-0.07  0.47  0.08]
 [  nan   nan   nan]
 [-0.07  0.95  0.1 ]
 [-0.16  1.41  0.15]
 [  nan   nan   nan]
 [-0.07  0.95 -0.1 ]
 [-0.05  1.14  0.15]
 [ 0.04  0.91  0.17]
 [-0.07  0.47 -0.08]
 [-0.16  1.41 -0.15]
 [-0.07 -0.    0.08]
 [-0.07 -0.   -0.08]]


In [ ]:
# Debug
np.savetxt("tmp/Q_qpIK_vel.csv",Q,delimiter=',')

In [ ]:
Q_gd = np.loadtxt("tmp/Q_qpIK.csv", delimiter=',')
Q_qp = np.loadtxt("tmp/Q_qpIK_vel.csv", delimiter=',')

import matplotlib.pyplot as plt
angles = []
for i in range(s._nimble.getNumJoints()):
    j = s._nimble.getJoint(i)
    for k in range(j.getNumDofs()):
        angles.append(j.getName()+'_'+j.getDofName(k))

i = 24
plt.title(angles[i]+" ("+str(i)+")")
plt.plot(np.array(Q_gd)[:,i]*180/np.pi,label="qp no vel")
plt.plot(np.array(Q_qp)[:,i]*180/np.pi,label="qp vel")
plt.legend()
plt.show()

In [ ]:
# What type of joint is 30 and 40?
J = s._nimble.getJoint("scapulothoracic_r")
J.getType()
# J.getInitialPositions()

In [ ]:
import pandas as pd
from Skeleton import Skeleton,DynamicSkeleton
import nimblephysics as nimble
import numpy as np
import json
import time
kps =  ['RKnee', 'LWrist', 'RHip', 'RShoulder',  'LElbow', 'LHip', 'RElbow', 'RWrist', 'LKnee', 'LShoulder', 'RAnkle', 'LAnkle']
s12 = Skeleton('/home/rmhri/markerless-human-perception/src/hpe/src/Biomechanical-Model/BODY12.xml')
s = DynamicSkeleton(config='/home/rmhri/markerless-human-perception/src/hpe/src/Biomechanical-Model/BODY15_constrained_3D.xml', \
    osim_file='/media/rmhri/mrom/data/skel_models_v1.1/bsm.osim')
# Debug qpIK
df = pd.read_csv('/home/rmhri/mrom/data/S9_Directions.60457274_gt.csv')
t = 0
Q = []
P = []
print("Starting qpIK:\n")
total_time = 0
for t in range(df.shape[0]):
    target = np.array([np.array(df.loc[t,[l+":X",l+":Y",l+":Z"]].to_numpy()) for l in kps]).squeeze()/1000
        
    s12.load_from_numpy(target.reshape(-1,3),kps)
    s.load_from_BODY12(s12)
    s.scale()
    begin = time.time()
    # s.qpIK(100)
    print(t)
    if t == 0:
        s.qpIK(100)
    else:
        s.qpIK(100,0.02)
    total_time += time.time()-begin
    
    Q.append(s._nimble.getPositions())
    P.append(s.to_numpy())

print(total_time/df.shape[0])
    
Q = np.array(Q)
P = np.array(P)



In [ ]:
# Decomposing nimble forward kinematics
pos = s._nimble.getJointWorldPositions(s.joints)

# s._nimble.getPositions()

J = s._nimble.getJoint(1)
# print(pos[0:3])

transform = J.getParentBodyNode()
# transform.
print(transform.rotation(),transform.translation())
# trans = transform.translate()

In [ ]:
import matplotlib.pyplot as plt


angles = []
for i in range(s._nimble.getNumJoints()):
    j = s._nimble.getJoint(i)
    for k in range(j.getNumDofs()):
        angles.append(j.getName()+'_'+j.getDofName(k))
    
i = 20
    
plt.plot(np.array(Q)[:,i],label=angles[i])
plt.legend()
plt.show()


In [ ]:
from Skeleton import Skeleton,DynamicSkeleton
import numpy as np
s = DynamicSkeleton(config='/home/rmhri/markerless-human-perception/src/hpe/src/Biomechanical-Model/BODY15_constrained_3D.xml', \
    osim_file='/home/rmhri/markerless-human-perception/src/hpe/src/Biomechanical-Model/data/skel_models_v1.1/bsm.osim')

RCAJ = np.array([0.015,-0.035,-0.02])
RHGT = np.array([-0.05,0,0])
LCAJ = np.array([0.015,-0.035,0.02])
LHGT = np.array([-0.05,0,0])


scapula = s._nimble.getBodyNode('scapula_l')
transform = scapula.getWorldTransform()
gui.nativeAPI().renderSkeleton(s._nimble)
gui.nativeAPI().createSphere(key="LCAJ", radii=np.array([0.005,0.005,0.005]),pos=transform.multiply(LCAJ),color=np.array([255,0,0,1]))
gui.nativeAPI().createSphere(key="LHGT", radii=np.array([0.005,0.005,0.005]),pos=transform.multiply(LHGT),color=np.array([255,0,0,1]))
scapula = s._nimble.getBodyNode('scapula_r')
transform = scapula.getWorldTransform()
gui.nativeAPI().createSphere(key="RCAJ", radii=np.array([0.005,0.005,0.005]),pos=transform.multiply(RCAJ),color=np.array([255,0,0,1]))
gui.nativeAPI().createSphere(key="RHGT", radii=np.array([0.005,0.005,0.005]),pos=transform.multiply(RHGT),color=np.array([255,0,0,1]))